In [1]:
import os
import pandas as pd
import numpy as np
import time
import gc
import pickle

In [2]:
from process import assign_atoms_index, get_pickup

In [3]:
FOLDER = '../../data_kaggle/champs/'
OUTPUT = FOLDER + 'out/'
PROCESSED_NO = 0
PROCESSED = '20190703_dist_bond_dir_train_temp{}.pickle'.format(PROCESSED_NO)

In [4]:
# df_mulliken_charges = pd.read_csv(FOLDER + 'mulliken_charges.csv')
# df_sample =  pd.read_csv(FOLDER + 'sample_submission.csv')
# df_magnetic_shielding_tensors = pd.read_csv(FOLDER + 'magnetic_shielding_tensors.csv')
df = pd.read_csv(FOLDER + 'train.csv')
# df_test = pd.read_csv(FOLDER + 'test.csv')
# df_dipole_moments = pd.read_csv(FOLDER + 'dipole_moments.csv')
# df_potential_energy = pd.read_csv(FOLDER + 'potential_energy.csv')
df_structures = pd.read_csv(FOLDER + 'structures.csv')
# df_scalar_coupling_contributions = pd.read_csv(FOLDER + 'scalar_coupling_contributions.csv')

In [5]:
df_structures_idx = df_structures.set_index('molecule_name')
df_idx = df.set_index('molecule_name')

In [6]:
mols = df['molecule_name'].unique()

In [7]:
num = 7
dist_mat = np.zeros([0, num*5*5])
num_div = len(mols) // 4
atoms_idx = np.zeros([0], dtype=np.int32)
molecule_names = np.empty([0])

start = time.time()

for mol in mols[:num_div]:
    assigned_idxs = assign_atoms_index(df_idx, mol)
    dist_mat_mole = get_pickup(df_idx, df_structures_idx, mol, num_pickup=num)
    mol_name_arr = [mol] * len(assigned_idxs) 
    
    molecule_names = np.hstack([molecule_names, mol_name_arr])
    atoms_idx = np.hstack([atoms_idx, assigned_idxs])
    dist_mat = np.vstack([dist_mat, dist_mat_mole])
    
col_name_list = []
atoms = ['H', 'C', 'N', 'O', 'F']
for a in atoms:
    for n in range(num):
        col_name_list.append('dist_{}_{}'.format(a, n))
    for n in range(num):
        col_name_list.append('angle_{}_{}'.format(a, n))
    for n in range(num):
        col_name_list.append('orientation_{}_{}'.format(a, n))
    for n in range(num):
        col_name_list.append('bond_cos2_{}_{}'.format(a, n))
    for n in range(num):
        col_name_list.append('bond_cos3_{}_{}'.format(a, n))
        
se_mole = pd.Series(molecule_names, name='molecule_name')
se_atom_idx = pd.Series(atoms_idx, name='atom_index').astype('int32')
df_dist = pd.DataFrame(dist_mat, columns=col_name_list).astype('float32')
df_distance = pd.concat([se_mole, se_atom_idx,df_dist], axis=1)

elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

/home/ryoji/Documents/git/kaggle/champs/process.py:60: RuntimeWarning: invalid value encountered in true_divide
  out = (proj_1*proj_2).sum(axis=1) / np.linalg.norm(proj_1, axis=1) / np.linalg.norm(proj_2, axis=1)


elapsed_time:5.056297540664673[sec]


In [8]:
for i in range(num):
    del df_distance['bond_cos2_H_{}'.format(i)]
    del df_distance['bond_cos2_O_{}'.format(i)]
    del df_distance['bond_cos2_F_{}'.format(i)]
    del df_distance['bond_cos3_H_{}'.format(i)]
    del df_distance['bond_cos3_O_{}'.format(i)]
    del df_distance['bond_cos3_F_{}'.format(i)]

In [9]:
nums = [num, num, 5, 4, 3]
for a, n in zip(atoms, nums):
    for i in range(n, num):
        del df_distance['dist_{}_{}'.format(a, i)]
        del df_distance['angle_{}_{}'.format(a, i)]
        del df_distance['orientation_{}_{}'.format(a, i)]

In [10]:
del_list = ['bond_cos2_N_5', 'bond_cos2_N_6', 'bond_cos3_N_5', 'bond_cos3_N_6']
for d in del_list:
    del df_distance[d]

In [11]:
df_distance.shape

(315239, 104)

In [12]:
df_distance.to_pickle(OUTPUT + PROCESSED)

In [13]:
df_distance.head()

,molecule_name,atom_index,dist_H_0,dist_H_1,dist_H_2,dist_H_3,dist_H_4,dist_H_5,dist_H_6,angle_H_0,...,orientation_O_3,dist_F_0,dist_F_1,dist_F_2,angle_F_0,angle_F_1,angle_F_2,orientation_F_0,orientation_F_1,orientation_F_2
0,dsgdb9nsd_000001,0,0.915796,0.915795,0.915792,0.91579,0.0,0.0,0.0,0.816496,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,dsgdb9nsd_000001,1,0.560815,0.560806,0.560803,0.00000,0.0,0.0,0.0,0.816482,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,dsgdb9nsd_000001,2,0.560815,0.560806,0.560803,0.00000,0.0,0.0,0.0,0.816483,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,dsgdb9nsd_000001,3,0.560806,0.560806,0.560803,0.00000,0.0,0.0,0.0,0.816483,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,dsgdb9nsd_000001,4,0.560806,0.560806,0.560803,0.00000,0.0,0.0,0.0,0.816496,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
